In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chinese-couplets/couplet/vocabs
/kaggle/input/chinese-couplets/couplet/test/out.txt
/kaggle/input/chinese-couplets/couplet/test/in.txt
/kaggle/input/chinese-couplets/couplet/test/.in.txt.swp
/kaggle/input/chinese-couplets/couplet/test/.out.txt.swp
/kaggle/input/chinese-couplets/couplet/train/out.txt
/kaggle/input/chinese-couplets/couplet/train/in.txt
/kaggle/input/seq2seq-vocab/seq2seq_state.bin
/kaggle/input/seq2seq-vocab/vocab.bin


In [2]:
file_in='/kaggle/input/chinese-couplets/couplet/test/in.txt'
file_out='/kaggle/input/chinese-couplets/couplet/test/out.txt'

In [3]:
###读取数据
try:
    enc_data = []
    dec_data = []

    with open(file_in, 'r', encoding='utf-8') as f1, open(file_out, 'r', encoding='utf-8') as f2:
        for line1, line2 in zip(f1, f2):
            enc_data.append(line1.strip())
            dec_data.append(line2.strip())
            
except FileNotFoundError:
    print("文件未找到，请检查文件路径。")
except Exception as e:
    print(f"读取文件时出现错误: {e}")

In [4]:
enc_data[:10]

['腾 飞 上 铁 ， 锐 意 改 革 谋 发 展 ， 勇 当 千 里 马',
 '风 弦 未 拨 心 先 乱',
 '花 梦 粘 于 春 袖 口',
 '晋 世 文 章 昌 二 陆',
 '一 句 相 思 吟 岁 月',
 '几 树 梅 花 数 竿 竹',
 '未 舍 东 江 开 口 咏',
 '彩 屏 如 画 ， 望 秀 美 崤 函 ， 花 团 锦 簇',
 '玉 液',
 '珍 藏 惟 有 诗 三 卷']

In [5]:
dec_data[:10]

['和 谐 南 供 ， 安 全 送 电 保 畅 通 ， 争 做 领 头 羊',
 '夜 幕 已 沉 梦 更 闲',
 '莺 声 溅 落 柳 枝 头',
 '魏 家 词 赋 重 三 曹',
 '千 杯 美 酒 醉 风 情',
 '一 潭 秋 水 半 屏 山',
 '且 施 妙 手 点 睛 来',
 '短 信 报 春 ， 喜 和 谐 社 会 ， 物 阜 民 康',
 '金 波',
 '馈 赠 则 无 画 半 帧']

In [6]:
import torch
import pickle
from seq2seqattencat import Seq2Seq



In [7]:
# 加载训练好的模型和词典
state_dict = torch.load('/kaggle/input/seq2seq-vocab/seq2seq_state.bin')
with open('/kaggle/input/seq2seq-vocab/vocab.bin','rb') as f:
    evoc,dvoc = pickle.load(f)

model = Seq2Seq(
    enc_emb_size=len(evoc),
    dec_emb_size=len(dvoc),
    emb_dim=100,
    hidden_size=120,
    dropout=0.5,
)
model.load_state_dict(state_dict)


/tmp/ipykernel_31/2571343456.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('/kaggle/input/seq2seq-vocab/seq2seq_state.bin')
/usr/local/lib/pyth

<All keys matched successfully>

In [55]:
def infer(enc_input):
    # 创建解码器反向字典
    dvoc_inv = {v:k for k,v in dvoc.items()}
    
    # 用户输入
    # enc_input = "玉 液"
    # enc_input = "What I'm about to say is strictly between you and me"
    # enc_input = "I used to go swimming in the sea when I was a child"
    # enc_input = enc_input.replace(',','').replace('.','').replace('!','').replace('?','')
    
    enc_idx = torch.tensor([[evoc[tk] for tk in enc_input.split(" ")]])
    
    
    # 推理
    # 最大解码长度
    max_dec_len = 50
    
    model.eval()
    with torch.no_grad():
        # 编码器
        # hidden_state = model.encoder(enc_idx)
        hidden_state, enc_outputs = model.encoder(enc_idx)  # attention
        
        # 解码器输入 shape [1,1]
        dec_input = torch.tensor([[dvoc['BOS']]])
        
        # 循环decoder
        dec_tokens = []
    while True:
        if len(dec_tokens) >= enc_idx.shape[1]:
            break
        # 解码器 
        # logits: [1,1,dec_voc_size]
        # logits,hidden_state = model.decoder(dec_input, hidden_state)
        logits,hidden_state = model.decoder(dec_input, hidden_state, enc_outputs) ##ATTENTION 
        
        # 下个token index
        next_token = torch.argmax(logits, dim=-1)
        if dvoc_inv[next_token.squeeze().item()] == 'EOS':
            break
        # 收集每次token_index 【解码集合】
        dec_tokens.append(next_token.squeeze().item())
        # decoder的下一个输入 = token_index
        dec_input = next_token
        hidden_state = hidden_state.view(1, -1)
    
    # 输出解码结果
    return ' '.join([dvoc_inv[tk] for tk in dec_tokens])

In [57]:
print("上 联   ：",enc_data[0])
print("下 联   ：",dec_data[0])
print("推理下联 ：",infer(enc_data[0]))

上 联   ： 腾 飞 上 铁 ， 锐 意 改 革 谋 发 展 ， 勇 当 千 里 马
下 联   ： 和 谐 南 供 ， 安 全 送 电 保 畅 通 ， 争 做 领 头 羊
推理下联 ： 改 革 新 潮 ， 激 情 燃 彩 旗 红 红 ， 又 做 万 家 春


In [58]:
index=0
for enc,dec in zip(enc_data,dec_data):
    print("上 联   ：",enc)
    print("下 联   ：",dec)
    print("推理下联 ：",infer(enc))
    print()
    index+=1
    if index==100:
        break

上 联   ： 腾 飞 上 铁 ， 锐 意 改 革 谋 发 展 ， 勇 当 千 里 马
下 联   ： 和 谐 南 供 ， 安 全 送 电 保 畅 通 ， 争 做 领 头 羊
推理下联 ： 改 革 新 潮 ， 激 情 燃 彩 旗 红 红 ， 又 做 万 家 春

上 联   ： 风 弦 未 拨 心 先 乱
下 联   ： 夜 幕 已 沉 梦 更 闲
推理下联 ： 月 色 犹 思 梦 已 圆

上 联   ： 花 梦 粘 于 春 袖 口
下 联   ： 莺 声 溅 落 柳 枝 头
推理下联 ： 风 情 动 在 月 牙 梢

上 联   ： 晋 世 文 章 昌 二 陆
下 联   ： 魏 家 词 赋 重 三 曹
推理下联 ： 文 章 武 穆 振 三 秦

上 联   ： 一 句 相 思 吟 岁 月
下 联   ： 千 杯 美 酒 醉 风 情
推理下联 ： 千 秋 寂 寞 赋 春 秋

上 联   ： 几 树 梅 花 数 竿 竹
下 联   ： 一 潭 秋 水 半 屏 山
推理下联 ： 一 枝 梅 朵 几 枝 梅

上 联   ： 未 舍 东 江 开 口 咏
下 联   ： 且 施 妙 手 点 睛 来
推理下联 ： 自 来 南 岳 展 胸 怀

上 联   ： 彩 屏 如 画 ， 望 秀 美 崤 函 ， 花 团 锦 簇
下 联   ： 短 信 报 春 ， 喜 和 谐 社 会 ， 物 阜 民 康
推理下联 ： 金 谷 似 歌 舞 ， 看 和 谐 盛 世 ， 春 满 人

上 联   ： 玉 液
下 联   ： 金 波
推理下联 ： 汤 汤

上 联   ： 珍 藏 惟 有 诗 三 卷
下 联   ： 馈 赠 则 无 画 半 帧
推理下联 ： 世 事 不 知 酒 一 杯

上 联   ： 岁 尽 方 知 时 日 快
下 联   ： 钱 空 始 觉 世 情 薄
推理下联 ： 年 来 更 觉 后 年 长

上 联   ： 路 不 拾 遗 ， 夜 不 闭 户
下 联   ： 一 夫 当 关 ， 万 夫 莫 开
推理下联 ： 时 俱 聚 会 ， 时 俱 归 心

上 联   ： 国 家 至 上 ， 讨 袁 抗 日 ， 丰 碑 两 座 彰 军 史
下 联   ： 民 族 当 头 ， 征 缅 战 台 ， 碧 血 一 腔 付 义 旗
推理下联 ： 史 册 长 留 ， 继 往 开 来 ， 伟